In [1]:
import os, sys
import numpy as np
import pandas as pd
import copy

things_root = '/user_data/mmhender/stimuli/things/'
project_root = '/user_data/mmhender/featsynth/'
stimuli_folder = '/user_data/mmhender/stimuli/featsynth/images_comb64/'
ecoset_info_path = '/user_data/mmhender/stimuli/ecoset_info/'

sys.path.insert(0, '/user_data/mmhender/featsynth/code/')
from utils import expt_utils

expt_name = 'fmri_pilot1'

In [4]:
np.log(1)/np.log(1)

0.0

In [9]:
np.logspace(1, 5, num=40, base=1.5)

array([1.5       , 1.56369447, 1.6300936 , 1.69931224, 1.7714701 ,
       1.846692  , 1.92510805, 2.00685388, 2.09207088, 2.18090645,
       2.27351424, 2.37005443, 2.47069401, 2.57560705, 2.684975  ,
       2.79898705, 2.91784038, 3.04174058, 3.17090196, 3.30554791,
       3.44591133, 3.592235  , 3.74477201, 3.9037862 , 4.0695526 ,
       4.24235794, 4.4225011 , 4.61029369, 4.8060605 , 5.01014016,
       5.22288565, 5.44466495, 5.67586166, 5.91687567, 6.16812385,
       6.43004078, 6.70307949, 6.98771223, 7.28443133, 7.59375   ])

In [19]:
itis = np.logspace(1, 10, num=40, base=1.1)
print(itis)
np.sum(itis)

[1.1        1.12446215 1.14946831 1.17503055 1.20116126 1.22787307
 1.25517891 1.28309198 1.3116258  1.34079416 1.37061117 1.40109126
 1.43224918 1.4641     1.49665913 1.52994232 1.56396567 1.59874564
 1.63429906 1.67064313 1.70779543 1.74577394 1.78459702 1.82428346
 1.86485247 1.90632366 1.9487171  1.9920533  2.03635322 2.0816383
 2.12793045 2.17525205 2.22362601 2.27307572 2.32362511 2.37529863
 2.42812129 2.48211864 2.53731679 2.59374246]


69.76348780912187

In [16]:
itis = np.linspace(1, 5, 40)
print(itis)
np.sum(itis)

[1.         1.1025641  1.20512821 1.30769231 1.41025641 1.51282051
 1.61538462 1.71794872 1.82051282 1.92307692 2.02564103 2.12820513
 2.23076923 2.33333333 2.43589744 2.53846154 2.64102564 2.74358974
 2.84615385 2.94871795 3.05128205 3.15384615 3.25641026 3.35897436
 3.46153846 3.56410256 3.66666667 3.76923077 3.87179487 3.97435897
 4.07692308 4.17948718 4.28205128 4.38461538 4.48717949 4.58974359
 4.69230769 4.79487179 4.8974359  5.        ]


120.0

In [2]:
rndseed=234445

# using here the same set of categories that we chose from ecoset.
# 64 basic categories in 8 superordinate groups.
fn = os.path.join(ecoset_info_path, 'categ_use_ecoset.npy')
info = np.load(fn, allow_pickle=True).item()
basic_names = list(info['binfo'].keys())
super_names = list(info['sinfo'].keys())

# load info about which images to use
# made this in make_featsynth_images_comb64.py
fn = os.path.join(stimuli_folder, 'synth_losses_all.npy')
print(fn)
l = np.load(fn, allow_pickle=True).item()

categ_sets = [super_names]
concept_sets = [[info['sinfo'][sname]['basic_names'] for sname in super_names]]


# figure out trial conditions/counts
n_categ_use = len(categ_sets[0])
n_concepts_use = len(concept_sets[0][0])
# n_ex_use = len(image_name_sets[0][0][0])

# image types are original or "synth" from different DNN layers
n_layers=4
n_image_types = n_layers+1 

cue_levels = ['basic', 'super']
n_cue_levels = len(cue_levels); # basic or superordinate

# this is per session
n_trials_per_concept = n_image_types * n_cue_levels
# assert(n_trials_per_concept==n_ex_use)

# creating up to three sessions. Not sure if we will run all these, 2 might be enough.
n_sessions = 3;

# how many unique exemplars will we need for all sessions?
n_ex_use = n_trials_per_concept * n_sessions 

n_trials_per_session = n_categ_use * n_concepts_use * n_trials_per_concept

# this is all per session
n_runs = 16;
assert(np.mod(n_trials_per_session, n_runs)==0)
n_trials_per_run = int(n_trials_per_session/n_runs);

# each run will have 4 mini-blocks of 10 trials each
n_trials_mini = 10

assert(np.mod(n_trials_per_run, n_trials_mini)==0)
n_mini_per_run = int(n_trials_per_run/n_trials_mini)

n_mini_total = n_mini_per_run * n_runs

n_trials_per_categ = n_trials_per_run # this just happens to work out this way
n_mini_per_categ = int(n_trials_per_categ / n_trials_mini)

np.random.seed(rndseed)

categ_use = categ_sets[0]
categ_use = [c.replace(' ', '_') for c in categ_use]
print(categ_use)
concepts_use = concept_sets[0]
# image_names_use = image_name_sets[cb]

# make 100 different randomized orders for this same image set
n_random_orders=2;


# this will be a list over all randomized orders
trial_info_list = []


/user_data/mmhender/stimuli/featsynth/images_comb64/synth_losses_all.npy
['insect', 'mammal', 'vegetable', 'fruit', 'tool', 'musical_instrument', 'furniture', 'vehicle']


In [5]:
n_trials_per_session

640

In [8]:
640 / 8 / 2

40.0

In [4]:
rand = 0;

 # this will be a list for this random order, all sessions
trial_info_allsess = []

# this is a coinflip that will be used to pick which task to start with
odd_even = int(np.random.normal(0,1,1)>0)
print(odd_even)

ex_each_sess = dict()
for bname in basic_names:
    # this is the order of the exemplars to use, sorted by synth losses
    # picking the best ones to use for the experiment 
    order = l[bname]['order'][0:n_ex_use]
    # each of these exemplars can be used exactly once in whole experiment 
    # (across all image types).
    # going to randomize how these exemplars are assigned to sessions. 
    rndorder = order[np.random.permutation(len(order))]
    ex_each_sess[bname] = np.reshape(rndorder, [n_trials_per_concept, n_sessions])
     

1


In [6]:
sess = 0;
print('making rand order %d, session %d'%(rand, sess))
                
    
trial_info = pd.DataFrame({'trial_num_overall': np.zeros((n_trials_per_session,)), 
           'trial_in_run': np.zeros((n_trials_per_session,)), 
           'trial_in_miniblock': np.zeros((n_trials_per_session,)), 
            'run_number': np.zeros((n_trials_per_session,)), 
            'miniblock_number_overall': np.zeros((n_trials_per_session,)), 
            'miniblock_number_in_run': np.zeros((n_trials_per_session,)), 
            'session_number': np.zeros((n_trials_per_session,)), 
            'image_set_num': np.zeros((n_trials_per_session,)), 
            'random_order_number': np.zeros((n_trials_per_session,)), 
           'categ_ind': np.zeros((n_trials_per_session,)),
          'concept_ind': np.zeros((n_trials_per_session,)),
          'super_name': np.zeros((n_trials_per_session,)),
          'basic_name': np.zeros((n_trials_per_session,)),
          'ex_num': np.zeros((n_trials_per_session,)),
          'ex_num_actual': np.zeros((n_trials_per_session,)),
          'image_type_num': np.zeros((n_trials_per_session,)),
          'image_type': np.zeros((n_trials_per_session,)),
          'image_name': np.zeros((n_trials_per_session,)),
          'cue_level_num': np.zeros((n_trials_per_session,)),
          'cue_level': np.zeros((n_trials_per_session,)),
          'cue_name': np.zeros((n_trials_per_session,)),
          'distractor_name': np.zeros((n_trials_per_session,)),
          'left_name': np.zeros((n_trials_per_session,)),
          'right_name': np.zeros((n_trials_per_session,)),
          'correct_resp': np.zeros((n_trials_per_session,)), 
          })

image_type_names = ['orig', 'pool1','pool2','pool3','pool4']


tt=-1
for ca in range(n_categ_use):

    for co in range(n_concepts_use):

        bname = concepts_use[ca][co]
        
        ex = -1

        # this is the set of exemplars assigned to this session.
        # they will be randomly assigned to trial types
        # for example ex01 might be used for intact or pool4, but never both
        ex_use_list = ex_each_sess[bname][:,sess]
       
        for typ in range(n_image_types):
            for cue in range(n_cue_levels):

                tt+=1
                if np.mod(tt,100)==0:
                    print('proc trial %d of %d'%(tt, n_trials_per_session))

                trial_info['trial_num_overall'].iloc[tt] = tt
                trial_info['categ_ind'].iloc[tt] = ca
                trial_info['concept_ind'].iloc[tt] = co
                trial_info['super_name'].iloc[tt] = categ_use[ca]
                trial_info['basic_name'].iloc[tt] = concepts_use[ca][co].replace(' ', '_')

                trial_info['image_type_num'].iloc[tt] = typ
                trial_info['image_type'].iloc[tt] = image_type_names[typ]
                trial_info['cue_level_num'].iloc[tt] = cue
                trial_info['cue_level'].iloc[tt] = cue_levels[cue]

                trial_info['image_set_num'].iloc[tt] = 0
                trial_info['random_order_number'].iloc[tt] = rand
                trial_info['session_number'].iloc[tt] = sess

                ex += 1
                # this is the exemplar index within the experiment
                trial_info['ex_num'].iloc[tt] = ex

                # this is the actual number of the exemplar, 0-40
                ex_num_actual = ex_use_list[ex]
                trial_info['ex_num_actual'].iloc[tt] = ex_num_actual

                ex_name = 'ex%02d'%(ex_num_actual)
                
                if image_type_names[typ]=='orig':

                    trial_info['image_name'].iloc[tt] = os.path.join(bname, ex_name, 'orig.png')

                else:

                    trial_info['image_name'].iloc[tt] = os.path.join(bname, ex_name,
                                                                     'scramble_upto_%s.png'%\
                                                                     (image_type_names[typ]))
                   
  

making rand order 0, session 0
proc trial 0 of 640
proc trial 100 of 640
proc trial 200 of 640
proc trial 300 of 640
proc trial 400 of 640
proc trial 500 of 640
proc trial 600 of 640


In [7]:
trial_info

,trial_num_overall,trial_in_run,trial_in_miniblock,run_number,miniblock_number_overall,miniblock_number_in_run,session_number,image_set_num,random_order_number,categ_ind,...,image_type_num,image_type,image_name,cue_level_num,cue_level,cue_name,distractor_name,left_name,right_name,correct_resp
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,orig,beetle/ex08/orig.png,0.0,basic,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,orig,beetle/ex07/orig.png,1.0,super,0.0,0.0,0.0,0.0,0.0
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,pool1,beetle/ex10/scramble_upto_pool1.png,0.0,basic,0.0,0.0,0.0,0.0,0.0
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,pool1,beetle/ex16/scramble_upto_pool1.png,1.0,super,0.0,0.0,0.0,0.0,0.0
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,pool2,beetle/ex00/scramble_upto_pool2.png,0.0,basic,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,635.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,...,2.0,pool2,canoe/ex07/scramble_upto_pool2.png,1.0,super,0.0,0.0,0.0,0.0,0.0
636,636.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,...,3.0,pool3,canoe/ex29/scramble_upto_pool3.png,0.0,basic,0.0,0.0,0.0,0.0,0.0
637,637.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,...,3.0,pool3,canoe/ex23/scramble_upto_pool3.png,1.0,super,0.0,0.0,0.0,0.0,0.0
638,638.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,...,4.0,pool4,canoe/ex18/scramble_upto_pool4.png,0.0,basic,0.0,0.0,0.0,0.0,0.0


In [9]:

# figure out the sequence of which task happens when

# this value is always alternating across sessions
odd_even += 1
odd_even = np.mod(odd_even,2)
print(odd_even)

miniblock_list = np.arange(n_mini_total)
miniblock_run = np.repeat(np.arange(n_runs), n_mini_per_run)

# miniblock tasks will always alternate, within a run
miniblock_task = np.roll(np.tile(np.arange(n_cue_levels), [int(n_mini_total/2),]), odd_even)

# flip the ordering of task within the "odd" runs
# so that every other run starts with coarse/fine miniblock
for rr in np.arange(1, n_runs, 2):
    miniblock_task[miniblock_run==rr] = np.flipud(miniblock_task[miniblock_run==rr])

# decide which super-category is being tested on each run of the basic task
# for the super task, this isn't relevant
miniblock_supcat = np.zeros_like(miniblock_list)
miniblock_supcat[miniblock_task==1] = -1

# want to find a way of assigning super-categories to miniblocks
# so that we don't have the same super-category twice in a run
# use brute force randomization to find good sequence
good_seq = False
max_iter = 100
ii = 0;
while (not good_seq) and (ii < max_iter):

    supcat_seq = np.repeat(np.arange(n_categ_use), n_mini_per_categ)
    supcat_seq = supcat_seq[np.random.permutation(len(supcat_seq))]
    miniblock_supcat[miniblock_task==0] = supcat_seq

    un_cats = [np.unique(miniblock_supcat[(miniblock_task==0) & (miniblock_run==rr)]) \
               for rr in range(n_runs)]
    good_seq = np.all([len(u)==2 for u in un_cats])

    ii+=1
    
    
# now group the trials into runs based on which task is being done
            
# first basic-level tasks, within each superordinate   
for si, sname in enumerate(categ_use):

    for typ in range(n_image_types):

        trial_inds = np.array((trial_info['super_name']==sname) & \
                              (trial_info['cue_level']=='basic') & \
                              (trial_info['image_type_num']==typ))

        # name of actual basic-level category
        cue_names_actual = np.array(trial_info['basic_name'].iloc[trial_inds])

        # now assign the "distractor" names by shuffling real names
        distract_names = expt_utils.shuffle_nosame(cue_names_actual)

        trial_info['cue_name'].iloc[trial_inds] =  cue_names_actual
        trial_info['distractor_name'].iloc[trial_inds] = distract_names

        # randomly assign whether left or right side is the actual cue name 
        cue_on_left = np.mod(np.random.permutation(len(cue_names_actual)),2)==0
        left_name = copy.deepcopy(cue_names_actual)
        right_name = copy.deepcopy(distract_names)
        # switch half of them
        left_name[~cue_on_left] = distract_names[~cue_on_left]
        right_name[~cue_on_left] = cue_names_actual[~cue_on_left]

        # correct resp: 1 for left, 2 for right
        correct_resp = np.ones_like(cue_on_left).astype(int)
        correct_resp[~cue_on_left] = 2

        trial_info['left_name'].iloc[trial_inds] = left_name
        trial_info['right_name'].iloc[trial_inds] = right_name

        trial_info['correct_resp'].iloc[trial_inds] = correct_resp


    # now grab the trials for all image types, within this superordinate categ
    trial_inds = np.array((trial_info['super_name']==sname) & \
                              (trial_info['cue_level']=='basic'))

    # break all these into equal sized "mini-blocks" randomly
    # assigning to the block numbers that we already decided on previously
    miniblock_inds_use = np.repeat(miniblock_list[miniblock_supcat==si], n_trials_mini)
    run_inds_use = np.repeat(miniblock_run[miniblock_supcat==si], n_trials_mini)

    rand_order = np.random.permutation(len(miniblock_inds_use))
    miniblock_inds_use = miniblock_inds_use[rand_order]
    run_inds_use = run_inds_use[rand_order]

    # print(miniblock_inds_use )
    trial_info['run_number'].iloc[trial_inds] = run_inds_use
    trial_info['miniblock_number_overall'].iloc[trial_inds] = miniblock_inds_use
    trial_info['miniblock_number_in_run'].iloc[trial_inds] = np.mod(miniblock_inds_use, n_mini_per_run)


# now organizing the superordinate task runs
# looping over image types, things will be shuffled within image type
for typ in range(n_image_types):

    trial_inds = np.array((trial_info['cue_level']=='super') & \
                          (trial_info['image_type_num']==typ) 
                          )

    # print(np.sum(trial_inds))

    # actual cue names (superordinate)
    cue_names_actual = np.array(trial_info['super_name'].iloc[trial_inds])

    # creating the list of "distractor" names that are pseudorandom
    distract_names = [np.array(categ_use)[np.array(categ_use)!=categ] for categ in categ_use]
    distract_names = [d[np.random.permutation(len(d))] for d in distract_names]
    extras = expt_utils.shuffle_nosame(np.array(categ_use))
    distract_names = [list(d)+[e] for d, e in zip(distract_names, extras)]
    distract_names = np.concatenate(distract_names, axis=0)

    # check they meet all the criteria
    un, counts = np.unique(distract_names, return_counts=True)
    assert(np.all(counts==counts[0]))
    assert not np.any(distract_names==cue_names_actual)

    trial_info['cue_name'].iloc[trial_inds] =  cue_names_actual
    trial_info['distractor_name'].iloc[trial_inds] = distract_names


    # randomly assign whether left or right side is the actual cue name 
    n_each = [np.sum(cue_names_actual==categ) for categ in categ_use]
    cue_on_left = np.concatenate([np.mod(np.random.permutation(n),2)==0 for n in n_each], \
                                 axis=0)
    left_name = copy.deepcopy(cue_names_actual)
    right_name = copy.deepcopy(distract_names)
    # switch half of them
    left_name[~cue_on_left] = distract_names[~cue_on_left]
    right_name[~cue_on_left] = cue_names_actual[~cue_on_left]

    # correct resp: 1 for left, 2 for right
    correct_resp = np.ones_like(cue_on_left).astype(int)
    correct_resp[~cue_on_left] = 2

    trial_info['left_name'].iloc[trial_inds] = left_name
    trial_info['right_name'].iloc[trial_inds] = right_name

    trial_info['correct_resp'].iloc[trial_inds] = correct_resp

# now grab all the trials in superordinate task, all image types
trial_inds = np.array(trial_info['cue_level']=='super')

# break all these into equal sized "mini-blocks" randomly
# assigning to the block numbers that we already decided on previously
miniblock_inds_use = np.repeat(miniblock_list[miniblock_task==1], n_trials_mini)
run_inds_use = np.repeat(miniblock_run[miniblock_task==1], n_trials_mini)

rand_order = np.random.permutation(len(miniblock_inds_use))
miniblock_inds_use = miniblock_inds_use[rand_order]
run_inds_use = run_inds_use[rand_order]

# print(miniblock_inds_use)

trial_info['run_number'].iloc[trial_inds] = run_inds_use
trial_info['miniblock_number_overall'].iloc[trial_inds] = miniblock_inds_use
trial_info['miniblock_number_in_run'].iloc[trial_inds] = np.mod(miniblock_inds_use, n_mini_per_run)


# organize the trials by mini-block number
trial_info.sort_values(by='miniblock_number_overall', inplace=True)

# now shuffle within each mini-block
for mb in np.unique(trial_info['miniblock_number_overall']):

    inds = np.where(np.array(trial_info['miniblock_number_overall']==mb))[0] 
    shuff_order = np.random.permutation(len(inds))
    trial_info.iloc[inds] = trial_info.iloc[inds[shuff_order]]

# assign trial numbers 
trial_info['trial_in_miniblock'] = np.tile(np.arange(n_trials_mini), [n_mini_total,])
trial_info['trial_in_run'] = np.tile(np.arange(n_trials_per_run), [n_runs,])
trial_info['trial_num_overall'] = np.arange(n_trials_per_session)

trial_info.set_index(np.arange(n_trials_per_session))



0


,trial_num_overall,trial_in_run,trial_in_miniblock,run_number,miniblock_number_overall,miniblock_number_in_run,session_number,image_set_num,random_order_number,categ_ind,...,image_type_num,image_type,image_name,cue_level_num,cue_level,cue_name,distractor_name,left_name,right_name,correct_resp
0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,pool4,butterfly/ex05/scramble_upto_pool4.png,0.0,basic,butterfly,mosquito,butterfly,mosquito,1.0
1,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,pool3,ant/ex09/scramble_upto_pool3.png,0.0,basic,ant,moth,moth,ant,2.0
2,2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,pool1,butterfly/ex27/scramble_upto_pool1.png,0.0,basic,butterfly,beetle,butterfly,beetle,1.0
3,3,3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,pool3,caterpillar/ex33/scramble_upto_pool3.png,0.0,basic,caterpillar,beetle,beetle,caterpillar,2.0
4,4,4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,pool1,grasshopper/ex15/scramble_upto_pool1.png,0.0,basic,grasshopper,mosquito,mosquito,grasshopper,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,635,35,5,15.0,63.0,3.0,0.0,0.0,0.0,2.0,...,4.0,pool4,coconut/ex31/scramble_upto_pool4.png,0.0,basic,coconut,raspberry,raspberry,coconut,2.0
636,636,36,6,15.0,63.0,3.0,0.0,0.0,0.0,2.0,...,1.0,pool1,coconut/ex15/scramble_upto_pool1.png,0.0,basic,coconut,cherry,coconut,cherry,1.0
637,637,37,7,15.0,63.0,3.0,0.0,0.0,0.0,2.0,...,4.0,pool4,pear/ex18/scramble_upto_pool4.png,0.0,basic,pear,banana,pear,banana,1.0
638,638,38,8,15.0,63.0,3.0,0.0,0.0,0.0,2.0,...,1.0,pool1,pomegranate/ex00/scramble_upto_pool1.png,0.0,basic,pomegranate,banana,pomegranate,banana,1.0


In [10]:
trial_info

,trial_num_overall,trial_in_run,trial_in_miniblock,run_number,miniblock_number_overall,miniblock_number_in_run,session_number,image_set_num,random_order_number,categ_ind,...,image_type_num,image_type,image_name,cue_level_num,cue_level,cue_name,distractor_name,left_name,right_name,correct_resp
28,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,pool4,butterfly/ex05/scramble_upto_pool4.png,0.0,basic,butterfly,mosquito,butterfly,mosquito,1.0
56,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,pool3,ant/ex09/scramble_upto_pool3.png,0.0,basic,ant,moth,moth,ant,2.0
10,2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,pool1,butterfly/ex27/scramble_upto_pool1.png,0.0,basic,butterfly,beetle,butterfly,beetle,1.0
32,3,3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,pool3,caterpillar/ex33/scramble_upto_pool3.png,0.0,basic,caterpillar,beetle,beetle,caterpillar,2.0
22,4,4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,pool1,grasshopper/ex15/scramble_upto_pool1.png,0.0,basic,grasshopper,mosquito,mosquito,grasshopper,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,635,35,5,15.0,63.0,3.0,0.0,0.0,0.0,2.0,...,4.0,pool4,coconut/ex31/scramble_upto_pool4.png,0.0,basic,coconut,raspberry,raspberry,coconut,2.0
206,636,36,6,15.0,63.0,3.0,0.0,0.0,0.0,2.0,...,1.0,pool1,coconut/ex15/scramble_upto_pool1.png,0.0,basic,coconut,cherry,coconut,cherry,1.0
188,637,37,7,15.0,63.0,3.0,0.0,0.0,0.0,2.0,...,4.0,pool4,pear/ex18/scramble_upto_pool4.png,0.0,basic,pear,banana,pear,banana,1.0
178,638,38,8,15.0,63.0,3.0,0.0,0.0,0.0,2.0,...,1.0,pool1,pomegranate/ex00/scramble_upto_pool1.png,0.0,basic,pomegranate,banana,pomegranate,banana,1.0


In [22]:
# double check everything
print('checking trial info')
check_trial_info(trial_info, concepts_use, categ_use)

trial_info_allsess += [trial_info]


checking trial info
['dessert' 'fruit' 'furniture' 'insect' 'musical_instrument' 'tool'
 'vegetable' 'vehicle'] [80 80 80 80 80 80 80 80]
['airplane' 'ant' 'apple' 'asparagus' 'axe' 'banana' 'bed' 'bee' 'beet'
 'beetle' 'bell' 'bench' 'broom' 'brownie' 'bus' 'butterfly' 'cabbage'
 'candy' 'canoe' 'car' 'caterpillar' 'chair' 'cherry' 'clarinet' 'coconut'
 'cookie' 'corn' 'couch' 'cupcake' 'cymbal' 'drum' 'grape' 'grasshopper'
 'guitar' 'hammer' 'knife' 'lamp' 'lettuce' 'milkshake' 'mosquito' 'moth'
 'motorcycle' 'muffin' 'onion' 'pastry' 'pea' 'pear' 'pencil' 'piano'
 'pie' 'pomegranate' 'pumpkin' 'raspberry' 'refrigerator' 'scissors'
 'ship' 'shovel' 'spoon' 'table' 'television' 'train' 'truck' 'trumpet'
 'violin'] [10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10]
[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.] [64 64 64 64 64 64 64 64 64 64]
[0. 1. 2. 3. 4.

In [52]:
bnames[0] = bnames[1]

In [58]:
enames[0] = enames[1]

In [61]:
un

array([['airplane', 'airplane', 'airplane', ..., 'violin', 'violin',
        'violin'],
       ['ex02', 'ex03', 'ex06', ..., 'ex34', 'ex36', 'ex39']],
      dtype='<U12')

In [25]:
# concatenating the trials for all sessions together
trial_info_allsess = pd.concat(trial_info_allsess)

# fix the indexing
trial_info_allsess.set_index(np.arange(n_trials_per_session * n_sessions))


ValueError: Length mismatch: Expected 1280 rows, received array of length 1920

In [12]:


def check_trial_info(ti, concepts_use, categ_use):

    # double checking all the trial attributes
    
    
    # check that we assigned the "correct response" correctly
    # 1 is left, 2 is right
    inds_check = np.array(ti['correct_resp']==1)
    assert(np.all(ti['cue_name'][inds_check]==ti['left_name'][inds_check]))
    inds_check = np.array(ti['correct_resp']==2)
    assert(np.all(ti['cue_name'][inds_check]==ti['right_name'][inds_check]))

    # cue and distractor name always different
    assert(not np.any(ti['cue_name']==ti['distractor_name']))

    # check that basic and super task are assigned correctly
    inds_check = np.array(ti['cue_level']=='basic')
    assert(np.all(np.isin(np.array(ti['cue_name'].iloc[inds_check]), concepts_use)))
    assert(np.all(np.isin(np.array(ti['distractor_name'].iloc[inds_check]), concepts_use)))

    # for each basic block checking that the cue and distractor names are all part of a single superord categ
    for mb in np.unique(ti['miniblock_number_overall'].iloc[inds_check]):
        inds_check_mb = inds_check & (np.array(ti['miniblock_number_overall']==mb))
        sname = np.array(ti['super_name'].iloc[inds_check_mb])
        assert(np.all(sname==sname[0]))
        sind = np.where(np.array(categ_use)==sname[0])[0][0]
        assert(np.all(np.isin(np.array(ti['distractor_name'].iloc[inds_check_mb]), concepts_use[sind])))
        assert(np.all(np.isin(np.array(ti['cue_name'].iloc[inds_check_mb]), concepts_use[sind])))

    # check superordinate names 
    inds_check = np.array(ti['cue_level']=='super')
    assert(np.all(np.isin(np.array(ti['cue_name'].iloc[inds_check]), categ_use)))
    assert(np.all(np.isin(np.array(ti['distractor_name'].iloc[inds_check]), categ_use)))

    # check to make sure individual attributes are distributed evenly across trials 
    n_trials_per_session = ti.shape[0]

    attr_check_even = ['super_name', 'basic_name', 'ex_num', 'image_type_num', \
                       'correct_resp', 'cue_level', 'run_number', \
                       'miniblock_number_overall', 'miniblock_number_in_run']
    for attr in attr_check_even:

        # should be an equal number of each thing 
        un, counts = np.unique(ti[attr], return_counts=True)
        print(un, counts)
        assert(np.all(counts==n_trials_per_session/len(un)))


    # TODO: check that the Actual exemplar numbers only used once per category
    # across all image types

    
    # check the counterbalancing over multiple attributes

    # there should be an equal number of trials in each of the combinations of these 
    # different attribute "levels". for example each combination of category/image type. 

    attr_balanced = [ti['categ_ind'], ti['concept_ind'], ti['image_type_num'], ti['cue_level']]
    attr_balanced_inds = np.array([np.unique(attr, return_inverse=True)[1] for attr in attr_balanced]).T

    n_levels_each = [len(np.unique(attr)) for attr in attr_balanced]
    n_combs_expected = np.prod(n_levels_each)
    n_repeats_expected = n_trials_per_session/n_combs_expected

    un_rows, counts = np.unique(attr_balanced_inds,axis=0, return_counts=True)

    assert(un_rows.shape[0]==n_combs_expected)
    assert(np.all(counts==n_repeats_expected))

    
    
    # check evenness of stuff for super task only

    inds_check = np.array(ti['cue_level']=='super')
    n_trials_per_session = np.sum(inds_check)

    attr_check_even = ['super_name', 'cue_name', 'image_type', 'distractor_name', 'correct_resp']

    for attr in attr_check_even:

        # should be an equal number of each thing 
        un, counts = np.unique(ti[attr].iloc[inds_check], return_counts=True)
        # print(un, counts)
        assert(np.all(counts==n_trials_per_session/len(un)))



    # check evenness of stuff for basic task only

    inds_check = np.array(ti['cue_level']=='basic')
    n_trials_per_session = np.sum(inds_check)

    attr_check_even = ['basic_name', 'cue_name', 'image_type', 'distractor_name', 'correct_resp']

    for attr in attr_check_even:

        # should be an equal number of each thing 
        un, counts = np.unique(ti[attr].iloc[inds_check], return_counts=True)
        # print(un, counts)
        assert(np.all(counts==n_trials_per_session/len(un)))

    # making sure that for the different superordinate tasks, 
    # the miniblocks all fall on different runs
    for si, sname in enumerate(categ_use):
        
        inds = (ti['cue_level']=='basic') & (ti['super_name']==sname)
        assert(len(np.unique(ti['run_number'][inds])) == 4)


In [38]:
# figure out the sequence of which task happens when

odd_even = int(np.random.normal(0,1,1)>0)
print(odd_even)

miniblock_list = np.arange(n_mini_total)
miniblock_run = np.repeat(np.arange(n_runs), n_mini_per_run)

# miniblock tasks will always alternate, within a run
miniblock_task = np.roll(np.tile(np.arange(n_cue_levels), [int(n_mini_total/2),]), odd_even)

# flip the ordering of task within the "odd" runs
# so that every other run starts with coarse/fine miniblock
for rr in np.arange(1, n_runs, 2):
    miniblock_task[miniblock_run==rr] = np.flipud(miniblock_task[miniblock_run==rr])

# decide which super-category is being tested on each run of the basic task
# for the super task, this isn't relevant
miniblock_supcat = np.zeros_like(miniblock_list)
miniblock_supcat[miniblock_task==1] = -1

# want to find a way of assigning super-categories to miniblocks
# so that we don't have the same super-category twice in a run
# use brute force randomization to find good sequence
good_seq = False
max_iter = 100
ii = 0;
while (not good_seq) and (ii < max_iter):

    supcat_seq = np.repeat(np.arange(n_categ_use), n_mini_per_categ)
    supcat_seq = supcat_seq[np.random.permutation(len(supcat_seq))]
    miniblock_supcat[miniblock_task==0] = supcat_seq

    un_cats = [np.unique(miniblock_supcat[(miniblock_task==0) & (miniblock_run==rr)]) \
               for rr in range(n_runs)]
    good_seq = np.all([len(u)==2 for u in un_cats])

    ii+=1
    

1


In [13]:
n_mini_per_run

4

In [11]:
len(miniblock_task)

64

In [12]:
64/4

16.0

In [39]:
# now group the trials into runs based on which task is being done
            
# first basic-level tasks, within each superordinate   
for si, sname in enumerate(categ_use):

    for typ in range(n_image_types):

        trial_inds = np.array((trial_info['super_name']==sname) & \
                              (trial_info['cue_level']=='basic') & \
                              (trial_info['image_type_num']==typ))

        # name of actual basic-level category
        cue_names_actual = np.array(trial_info['basic_name'].iloc[trial_inds])

        # now assign the "distractor" names by shuffling real names
        distract_names = expt_utils.shuffle_nosame(cue_names_actual)

        trial_info['cue_name'].iloc[trial_inds] =  cue_names_actual
        trial_info['distractor_name'].iloc[trial_inds] = distract_names

        # randomly assign whether left or right side is the actual cue name 
        cue_on_left = np.mod(np.random.permutation(len(cue_names_actual)),2)==0
        left_name = copy.deepcopy(cue_names_actual)
        right_name = copy.deepcopy(distract_names)
        # switch half of them
        left_name[~cue_on_left] = distract_names[~cue_on_left]
        right_name[~cue_on_left] = cue_names_actual[~cue_on_left]

        # correct resp: 1 for left, 2 for right
        correct_resp = np.ones_like(cue_on_left).astype(int)
        correct_resp[~cue_on_left] = 2

        trial_info['left_name'].iloc[trial_inds] = left_name
        trial_info['right_name'].iloc[trial_inds] = right_name

        trial_info['correct_resp'].iloc[trial_inds] = correct_resp


    # now grab the trials for all image types, within this superordinate categ
    trial_inds = np.array((trial_info['super_name']==sname) & \
                              (trial_info['cue_level']=='basic'))

    # break all these into equal sized "mini-blocks" randomly
    # assigning to the block numbers that we already decided on previously
    miniblock_inds_use = np.repeat(miniblock_list[miniblock_supcat==si], n_trials_mini)
    run_inds_use = np.repeat(miniblock_run[miniblock_supcat==si], n_trials_mini)

    rand_order = np.random.permutation(len(miniblock_inds_use))
    miniblock_inds_use = miniblock_inds_use[rand_order]
    run_inds_use = run_inds_use[rand_order]

    # print(miniblock_inds_use )
    trial_info['run_number'].iloc[trial_inds] = run_inds_use
    trial_info['miniblock_number_overall'].iloc[trial_inds] = miniblock_inds_use
    trial_info['miniblock_number_in_run'].iloc[trial_inds] = np.mod(miniblock_inds_use, n_mini_per_run)


In [49]:
n_sessions = 3
ex_use_session = np.repeat(np.arange(n_sessions), n_trials_per_concept)
ex_use_session

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2])

In [47]:
odd_even = 1

for ii in range(10):

    odd_even += 1
    odd_even = np.mod(odd_even,2)

    print(odd_even)

0
1
0
1
0
1
0
1
0
1


In [44]:
5 * 2 * 64

640

In [42]:
cue_names_actual

array(['ship', 'train', 'airplane', 'truck', 'car', 'bus', 'motorcycle',
       'canoe'], dtype=object)

In [43]:
trial_inds = np.array((trial_info['super_name']==sname) & \
                              (trial_info['cue_level']=='basic') & \
                              (trial_info['image_type_num']==typ))

trial_info.iloc[trial_inds]

,trial_num_overall,trial_in_run,trial_in_miniblock,run_number,miniblock_number_overall,miniblock_number_in_run,image_set_num,random_order_number,categ_ind,concept_ind,...,image_type_num,image_type,image_name,cue_level_num,cue_level,cue_name,distractor_name,left_name,right_name,correct_resp
568,568.0,0.0,0.0,5.0,22.0,2.0,0.0,0.0,7.0,0.0,...,4.0,pool4,ship/ex14/scramble_upto_pool4.png,0.0,basic,ship,train,ship,train,1.0
578,578.0,0.0,0.0,4.0,17.0,1.0,0.0,0.0,7.0,1.0,...,4.0,pool4,train/ex23/scramble_upto_pool4.png,0.0,basic,train,ship,ship,train,2.0
588,588.0,0.0,0.0,5.0,22.0,2.0,0.0,0.0,7.0,2.0,...,4.0,pool4,airplane/ex07/scramble_upto_pool4.png,0.0,basic,airplane,car,car,airplane,2.0
598,598.0,0.0,0.0,8.0,35.0,3.0,0.0,0.0,7.0,3.0,...,4.0,pool4,truck/ex23/scramble_upto_pool4.png,0.0,basic,truck,bus,truck,bus,1.0
608,608.0,0.0,0.0,4.0,17.0,1.0,0.0,0.0,7.0,4.0,...,4.0,pool4,car/ex25/scramble_upto_pool4.png,0.0,basic,car,airplane,airplane,car,2.0
618,618.0,0.0,0.0,4.0,17.0,1.0,0.0,0.0,7.0,5.0,...,4.0,pool4,bus/ex35/scramble_upto_pool4.png,0.0,basic,bus,truck,bus,truck,1.0
628,628.0,0.0,0.0,5.0,22.0,2.0,0.0,0.0,7.0,6.0,...,4.0,pool4,motorcycle/ex24/scramble_upto_pool4.png,0.0,basic,motorcycle,canoe,motorcycle,canoe,1.0
638,638.0,0.0,0.0,2.0,9.0,1.0,0.0,0.0,7.0,7.0,...,4.0,pool4,canoe/ex11/scramble_upto_pool4.png,0.0,basic,canoe,motorcycle,motorcycle,canoe,2.0


In [7]:

# now organizing the superordinate task runs
# looping over image types, things will be shuffled within image type
for typ in range(n_image_types):

    trial_inds = np.array((trial_info['cue_level']=='super') & \
                          (trial_info['image_type_num']==typ) 
                          )

    # print(np.sum(trial_inds))

    # actual cue names (superordinate)
    cue_names_actual = np.array(trial_info['super_name'].iloc[trial_inds])

    # creating the list of "distractor" names that are pseudorandom
    distract_names = [np.array(categ_use)[np.array(categ_use)!=categ] for categ in categ_use]
    distract_names = [d[np.random.permutation(len(d))] for d in distract_names]
    extras = expt_utils.shuffle_nosame(np.array(categ_use))
    distract_names = [list(d)+[e] for d, e in zip(distract_names, extras)]
    distract_names = np.concatenate(distract_names, axis=0)

    # check they meet all the criteria
    un, counts = np.unique(distract_names, return_counts=True)
    assert(np.all(counts==counts[0]))
    assert not np.any(distract_names==cue_names_actual)

    trial_info['cue_name'].iloc[trial_inds] =  cue_names_actual
    trial_info['distractor_name'].iloc[trial_inds] = distract_names


    # randomly assign whether left or right side is the actual cue name 
    n_each = [np.sum(cue_names_actual==categ) for categ in categ_use]
    cue_on_left = np.concatenate([np.mod(np.random.permutation(n),2)==0 for n in n_each], \
                                 axis=0)
    left_name = copy.deepcopy(cue_names_actual)
    right_name = copy.deepcopy(distract_names)
    # switch half of them
    left_name[~cue_on_left] = distract_names[~cue_on_left]
    right_name[~cue_on_left] = cue_names_actual[~cue_on_left]

    # correct resp: 1 for left, 2 for right
    correct_resp = np.ones_like(cue_on_left).astype(int)
    correct_resp[~cue_on_left] = 2

    trial_info['left_name'].iloc[trial_inds] = left_name
    trial_info['right_name'].iloc[trial_inds] = right_name

    trial_info['correct_resp'].iloc[trial_inds] = correct_resp

# now grab all the trials in superordinate task, all image types
trial_inds = np.array(trial_info['cue_level']=='super')

# break all these into equal sized "mini-blocks" randomly
# assigning to the block numbers that we already decided on previously
miniblock_inds_use = np.repeat(miniblock_list[miniblock_task==1], n_trials_mini)
run_inds_use = np.repeat(miniblock_run[miniblock_task==1], n_trials_mini)

rand_order = np.random.permutation(len(miniblock_inds_use))
miniblock_inds_use = miniblock_inds_use[rand_order]
run_inds_use = run_inds_use[rand_order]

# print(miniblock_inds_use)

trial_info['run_number'].iloc[trial_inds] = run_inds_use
trial_info['miniblock_number_overall'].iloc[trial_inds] = miniblock_inds_use
trial_info['miniblock_number_in_run'].iloc[trial_inds] = np.mod(miniblock_inds_use, n_mini_per_run)


In [8]:

# organize the trials by mini-block number
trial_info.sort_values(by='miniblock_number_overall', inplace=True)

# now shuffle within each mini-block
for mb in np.unique(trial_info['miniblock_number_overall']):

    inds = np.where(np.array(trial_info['miniblock_number_overall']==mb))[0] 
    shuff_order = np.random.permutation(len(inds))
    trial_info.iloc[inds] = trial_info.iloc[inds[shuff_order]]

# assign trial numbers 
trial_info['trial_in_miniblock'] = np.tile(np.arange(n_trials_mini), [n_mini_total,])
trial_info['trial_in_run'] = np.tile(np.arange(n_trials_per_run), [n_runs,])
trial_info['trial_num_overall'] = np.arange(n_trials_total)

trial_info.set_index(np.arange(n_trials_total))


,trial_num_overall,trial_in_run,trial_in_miniblock,run_number,miniblock_number_overall,miniblock_number_in_run,image_set_num,random_order_number,categ_ind,concept_ind,...,image_type_num,image_type,image_name,cue_level_num,cue_level,cue_name,distractor_name,left_name,right_name,correct_resp
0,0,0,0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,1.0,pool1,bell/ex32/scramble_upto_pool1.png,1.0,super,musical_instrument,furniture,musical_instrument,furniture,1.0
1,1,1,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2.0,pool2,pea/ex10/scramble_upto_pool2.png,1.0,super,vegetable,vehicle,vehicle,vegetable,2.0
2,2,2,2,0.0,0.0,0.0,0.0,0.0,2.0,3.0,...,4.0,pool4,apple/ex39/scramble_upto_pool4.png,1.0,super,fruit,musical_instrument,musical_instrument,fruit,2.0
3,3,3,3,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,3.0,pool3,ant/ex10/scramble_upto_pool3.png,1.0,super,insect,tool,tool,insect,2.0
4,4,4,4,0.0,0.0,0.0,0.0,0.0,5.0,6.0,...,4.0,pool4,clarinet/ex06/scramble_upto_pool4.png,1.0,super,musical_instrument,insect,musical_instrument,insect,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,635,35,5,15.0,63.0,3.0,0.0,0.0,0.0,6.0,...,0.0,orig,moth/ex16/orig.png,1.0,super,insect,tool,insect,tool,1.0
636,636,36,6,15.0,63.0,3.0,0.0,0.0,3.0,0.0,...,0.0,orig,candy/ex36/orig.png,1.0,super,dessert,musical_instrument,musical_instrument,dessert,2.0
637,637,37,7,15.0,63.0,3.0,0.0,0.0,6.0,0.0,...,0.0,orig,table/ex03/orig.png,1.0,super,furniture,fruit,fruit,furniture,2.0
638,638,38,8,15.0,63.0,3.0,0.0,0.0,5.0,2.0,...,1.0,pool1,piano/ex11/scramble_upto_pool1.png,1.0,super,musical_instrument,vehicle,musical_instrument,vehicle,1.0


In [28]:
inds = np.array((trial_info['cue_level']=='super') \
                & (trial_info['super_name']=='vehicle') \
               & (trial_info['image_type']=='orig'))

In [29]:
trial_info.iloc[inds]

,trial_num_overall,trial_in_run,trial_in_miniblock,run_number,miniblock_number_overall,miniblock_number_in_run,image_set_num,random_order_number,categ_ind,concept_ind,...,image_type_num,image_type,image_name,cue_level_num,cue_level,cue_name,distractor_name,left_name,right_name,correct_resp
85,5,5,5,0.0,0.0,0.0,0.0,0.0,7.0,4.0,...,0.0,orig,car/ex04/orig.png,1.0,super,vehicle,fruit,fruit,vehicle,2.0
485,85,5,5,2.0,8.0,0.0,0.0,0.0,7.0,1.0,...,0.0,orig,train/ex17/orig.png,1.0,super,vehicle,vegetable,vehicle,vegetable,1.0
561,89,9,9,2.0,8.0,0.0,0.0,0.0,7.0,0.0,...,0.0,orig,ship/ex20/orig.png,1.0,super,vehicle,insect,vehicle,insect,1.0
259,163,3,3,4.0,16.0,0.0,0.0,0.0,7.0,2.0,...,0.0,orig,airplane/ex10/orig.png,1.0,super,vehicle,musical_instrument,vehicle,musical_instrument,1.0
77,266,26,6,6.0,26.0,2.0,0.0,0.0,7.0,6.0,...,0.0,orig,motorcycle/ex20/orig.png,1.0,super,vehicle,furniture,furniture,vehicle,2.0
231,554,34,4,13.0,55.0,3.0,0.0,0.0,7.0,7.0,...,0.0,orig,canoe/ex34/orig.png,1.0,super,vehicle,furniture,vehicle,furniture,1.0
381,569,9,9,14.0,56.0,0.0,0.0,0.0,7.0,3.0,...,0.0,orig,truck/ex15/orig.png,1.0,super,vehicle,tool,tool,vehicle,2.0
297,586,26,6,14.0,58.0,2.0,0.0,0.0,7.0,5.0,...,0.0,orig,bus/ex31/orig.png,1.0,super,vehicle,dessert,dessert,vehicle,2.0


In [30]:
640 / 5 / 2 / 8

8.0

In [21]:
inds = np.array((trial_info['cue_level']=='basic') & (trial_info['super_name']=='vehicle'))

In [22]:
trial_info.iloc[inds]

,trial_num_overall,trial_in_run,trial_in_miniblock,run_number,miniblock_number_overall,miniblock_number_in_run,image_set_num,random_order_number,categ_ind,concept_ind,...,image_type_num,image_type,image_name,cue_level_num,cue_level,cue_name,distractor_name,left_name,right_name,correct_resp
604,90,10,0,2.0,9.0,1.0,0.0,0.0,7.0,2.0,...,3.0,pool3,airplane/ex08/scramble_upto_pool3.png,0.0,basic,airplane,car,airplane,car,1.0
638,91,11,1,2.0,9.0,1.0,0.0,0.0,7.0,7.0,...,4.0,pool4,canoe/ex11/scramble_upto_pool4.png,0.0,basic,canoe,motorcycle,motorcycle,canoe,2.0
624,92,12,2,2.0,9.0,1.0,0.0,0.0,7.0,0.0,...,3.0,pool3,ship/ex27/scramble_upto_pool3.png,0.0,basic,ship,motorcycle,motorcycle,ship,2.0
610,93,13,3,2.0,9.0,1.0,0.0,0.0,7.0,5.0,...,0.0,orig,bus/ex37/orig.png,0.0,basic,bus,motorcycle,bus,motorcycle,1.0
560,94,14,4,2.0,9.0,1.0,0.0,0.0,7.0,1.0,...,2.0,pool2,train/ex20/scramble_upto_pool2.png,0.0,basic,train,airplane,airplane,train,2.0
586,95,15,5,2.0,9.0,1.0,0.0,0.0,7.0,6.0,...,2.0,pool2,motorcycle/ex02/scramble_upto_pool2.png,0.0,basic,motorcycle,canoe,motorcycle,canoe,1.0
566,96,16,6,2.0,9.0,1.0,0.0,0.0,7.0,4.0,...,2.0,pool2,car/ex24/scramble_upto_pool2.png,0.0,basic,car,ship,ship,car,2.0
594,97,17,7,2.0,9.0,1.0,0.0,0.0,7.0,1.0,...,0.0,orig,train/ex02/orig.png,0.0,basic,train,canoe,canoe,train,2.0
574,98,18,8,2.0,9.0,1.0,0.0,0.0,7.0,0.0,...,0.0,orig,ship/ex13/orig.png,0.0,basic,ship,truck,truck,ship,2.0
570,99,19,9,2.0,9.0,1.0,0.0,0.0,7.0,3.0,...,2.0,pool2,truck/ex08/scramble_upto_pool2.png,0.0,basic,truck,bus,truck,bus,1.0


In [31]:
inds = np.array((trial_info['cue_level']=='basic') & (trial_info['super_name']=='vehicle') & \
        (trial_info['image_type']=='orig'))

In [32]:
trial_info.iloc[inds]

,trial_num_overall,trial_in_run,trial_in_miniblock,run_number,miniblock_number_overall,miniblock_number_in_run,image_set_num,random_order_number,categ_ind,concept_ind,...,image_type_num,image_type,image_name,cue_level_num,cue_level,cue_name,distractor_name,left_name,right_name,correct_resp
610,93,13,3,2.0,9.0,1.0,0.0,0.0,7.0,5.0,...,0.0,orig,bus/ex37/orig.png,0.0,basic,bus,motorcycle,bus,motorcycle,1.0
594,97,17,7,2.0,9.0,1.0,0.0,0.0,7.0,1.0,...,0.0,orig,train/ex02/orig.png,0.0,basic,train,canoe,canoe,train,2.0
574,98,18,8,2.0,9.0,1.0,0.0,0.0,7.0,0.0,...,0.0,orig,ship/ex13/orig.png,0.0,basic,ship,truck,truck,ship,2.0
582,170,10,0,4.0,17.0,1.0,0.0,0.0,7.0,4.0,...,0.0,orig,car/ex32/orig.png,0.0,basic,car,airplane,car,airplane,1.0
572,172,12,2,4.0,17.0,1.0,0.0,0.0,7.0,6.0,...,0.0,orig,motorcycle/ex03/orig.png,0.0,basic,motorcycle,bus,bus,motorcycle,2.0
616,228,28,8,5.0,22.0,2.0,0.0,0.0,7.0,3.0,...,0.0,orig,truck/ex31/orig.png,0.0,basic,truck,ship,truck,ship,1.0
576,351,31,1,8.0,35.0,3.0,0.0,0.0,7.0,2.0,...,0.0,orig,airplane/ex19/orig.png,0.0,basic,airplane,car,car,airplane,2.0
580,356,36,6,8.0,35.0,3.0,0.0,0.0,7.0,7.0,...,0.0,orig,canoe/ex30/orig.png,0.0,basic,canoe,train,canoe,train,1.0


In [26]:
inds = np.array((trial_info['basic_name']=='motorcycle'))

In [27]:
trial_info.iloc[inds]

,trial_num_overall,trial_in_run,trial_in_miniblock,run_number,miniblock_number_overall,miniblock_number_in_run,image_set_num,random_order_number,categ_ind,concept_ind,...,image_type_num,image_type,image_name,cue_level_num,cue_level,cue_name,distractor_name,left_name,right_name,correct_resp
586,95,15,5,2.0,9.0,1.0,0.0,0.0,7.0,6.0,...,2.0,pool2,motorcycle/ex02/scramble_upto_pool2.png,0.0,basic,motorcycle,canoe,motorcycle,canoe,1.0
626,171,11,1,4.0,17.0,1.0,0.0,0.0,7.0,6.0,...,3.0,pool3,motorcycle/ex32/scramble_upto_pool3.png,0.0,basic,motorcycle,ship,motorcycle,ship,1.0
572,172,12,2,4.0,17.0,1.0,0.0,0.0,7.0,6.0,...,0.0,orig,motorcycle/ex03/orig.png,0.0,basic,motorcycle,bus,bus,motorcycle,2.0
568,227,27,7,5.0,22.0,2.0,0.0,0.0,7.0,6.0,...,4.0,pool4,motorcycle/ex24/scramble_upto_pool4.png,0.0,basic,motorcycle,canoe,motorcycle,canoe,1.0
77,266,26,6,6.0,26.0,2.0,0.0,0.0,7.0,6.0,...,0.0,orig,motorcycle/ex20/orig.png,1.0,super,vehicle,furniture,furniture,vehicle,2.0
509,296,16,6,7.0,29.0,1.0,0.0,0.0,7.0,6.0,...,4.0,pool4,motorcycle/ex08/scramble_upto_pool4.png,1.0,super,vehicle,vegetable,vegetable,vehicle,2.0
375,313,33,3,7.0,31.0,3.0,0.0,0.0,7.0,6.0,...,2.0,pool2,motorcycle/ex11/scramble_upto_pool2.png,1.0,super,vehicle,fruit,vehicle,fruit,1.0
630,359,39,9,8.0,35.0,3.0,0.0,0.0,7.0,6.0,...,1.0,pool1,motorcycle/ex39/scramble_upto_pool1.png,0.0,basic,motorcycle,airplane,airplane,motorcycle,2.0
623,400,0,0,10.0,40.0,0.0,0.0,0.0,7.0,6.0,...,1.0,pool1,motorcycle/ex09/scramble_upto_pool1.png,1.0,super,vehicle,vegetable,vehicle,vegetable,1.0
517,480,0,0,12.0,48.0,0.0,0.0,0.0,7.0,6.0,...,3.0,pool3,motorcycle/ex36/scramble_upto_pool3.png,1.0,super,vehicle,fruit,vehicle,fruit,1.0


In [17]:
8*7

56

In [19]:
56/2

28.0

In [15]:
cue_names_actual

array(['ship', 'train', 'airplane', 'truck', 'car', 'bus', 'motorcycle',
       'canoe'], dtype=object)

In [16]:
distract_names

array(['airplane', 'bus', 'ship', 'car', 'truck', 'train', 'canoe',
       'motorcycle'], dtype=object)

In [14]:
e[i==0]

array([0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1.])

In [15]:
e[i==1]

array([2., 3., 2., 3., 2., 3., 2., 3., 2., 3., 2., 3., 2., 3., 2., 3., 2.,
       3., 2., 3., 2., 3., 2., 3., 2., 3., 2., 3., 2., 3., 2., 3., 2., 3.,
       2., 3., 2., 3., 2., 3., 2., 3., 2., 3., 2., 3., 2., 3., 2., 3., 2.,
       3., 2., 3., 2., 3., 2., 3., 2., 3., 2., 3., 2., 3., 2., 3., 2., 3.,
       2., 3., 2., 3., 2., 3., 2., 3., 2., 3., 2., 3., 2., 3., 2., 3., 2.,
       3., 2., 3., 2., 3., 2., 3., 2., 3., 2., 3., 2., 3., 2., 3., 2., 3.,
       2., 3., 2., 3., 2., 3., 2., 3., 2., 3., 2., 3., 2., 3., 2., 3., 2.,
       3., 2., 3., 2., 3., 2., 3., 2., 3.])